In [57]:
import pandas as pd 
import numpy as np
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['figure.dpi'] = 200
plt.style.use('seaborn')
from ipywidgets import interact, interactive, fixed, interact_manual,HBox, Layout,VBox,Label
import ipywidgets as widgets

In [3]:
age = np.arange(1,120)
df = {'genomic instability':5+0.01*np.power(age,2),
     'loss of proteostasis':5+0.05*np.power(age,1.5),
     'stem cell exhaustion':5+0.05*np.power(age,1.5),
     'epigenetic alterations':5+0.05*np.power(age,1.5),
     'mitochondrial dysfunction':5+0.05*np.power(age,1.5),
     'telomere attrition':5+0.05*np.power(age,1.5),
     'cellular senescence':5+np.power(age,1.05)
     }

interaction_term = 0

df = pd.DataFrame(df)
df['interaction'] = df.sum(axis=1)*0.1
reset_df = df.copy()

total = df.iloc[-1,:].sum()
threshold = total/2

In [159]:
# Here's where you can update how the interaction layer is calculated
# and how the negative variables can affect each variable in the dataframe

def dataframe_update(dataframe,v,t_y,neg,cmap):
    global threshold
    
    dataframe[v] = t_y
    
    # negative variable scaling: you can adjust the scaling for each variable
    # depending on literature
    if neg != 'none':
        if neg == 'smoking':
            variable_scaling = {'genomic instability':1.2,
                            'loss of proteostasis':1.2,
                            'stem cell exhaustion':1.2,
                            'epigenetic alterations':1.2,
                            'mitochondrial dysfunction':1.2,
                            'telomere attrition':1.2,
                            'cellular senescence':1.2}
        else:
            # this is just a placeholder. Copy paste the section under smoking to
            # customize scaling for each harmful variable
            variable_scaling = {key:1.0 for key in dataframe.columns.tolist()}
            
        harmed_dataframe = pd.DataFrame({key:dataframe[key].values*variable_scaling[key] for key in variable_scaling.keys()})
        harmed_dataframe['interaction'] = dataframe.iloc[:,:-1].sum(axis=1)*0.1 #interaction layer calculation

    
    dataframe['interaction'] = dataframe.iloc[:,:-1].sum(axis=1)*0.1 #interaction layer calculation
    
    if neg == 'none':
        ax = dataframe.plot.area(colormap=cmap,figsize=(10,5),linewidth=0)
        try:
            death_age = next(x[0] for x in enumerate(dataframe.sum(axis=1)) if x[1] > threshold)
        except:
            death_age = 120
    else:
        ax = harmed_dataframe.plot.area(colormap=cmap,figsize=(10,5),linewidth=0)
        try:
            death_age = next(x[0] for x in enumerate(harmed_dataframe.sum(axis=1)) if x[1] > threshold)
        except:
            death_age = 120
    return ax,death_age,dataframe

In [160]:
def plot_areas(intervention_age = 60, variable = 'genomic instability', harmful_variable = 'none', therapeutic_factor = 0.0, colormap='viridis', multiple = False, reset = False): 
    
    global df
    temp_df = reset_df.copy()
    
    if reset: df = reset_df.copy()
        
    temp_y = temp_df[variable].values 
    temp_y[intervention_age:] *= (1.0 - therapeutic_factor)
    
    global threshold
    
    if not multiple:
        ax,death_age,temp_df = dataframe_update(temp_df,variable,temp_y,harmful_variable,colormap)
    else:
        ax,death_age,df = dataframe_update(df,variable,temp_y,harmful_variable,colormap)
        
    ax.collections[-1].set_alpha(0.5)
    ax.collections[-1].set_hatch('\\')
    ax.collections[-1].set_color('gray')
    
    leg = plt.legend()
    leg.legendHandles[-1].set_alpha(0.5)
    
    plt.axhline(y=threshold, color='r', linestyle='--')
    if death_age<120:
        plt.scatter(death_age,threshold,marker='.',c='red',s=80)
        plt.text(death_age,threshold*1.05,str(death_age),fontsize=10,color='red')
        plt.text(2, threshold*1.05, 'YOU DIE AT:', fontsize = 10, color='red')
    else:
        plt.text(2, threshold*1.05, 'Longevity goal achieved', fontsize = 10, color='red')
    plt.xlim(0,len(age))
    plt.ylim(0,total)
    plt.xlabel('age')
    plt.ylabel('cellular damage')
    plt.title('Accumulated Cellular Damage')

In [161]:
df = reset_df.copy()

colormap = widgets.Dropdown(
    options = ['magma','viridis','plasma','inferno','cividis']
)
intervention_age = widgets.IntSlider(
    value = 60,
    min = 0,
    max = 120,
    step = 10
)
variable = widgets.Dropdown(
    options = df.columns.tolist()[:-1],
    value='epigenetic alterations'
)
harmful_variable = widgets.Dropdown(
    options = ['none','smoking','obesity','low socioeconomic status'],
    value='none'
)
therapeutic_factor = widgets.FloatSlider(
    value = 0.0, 
    min = 0.0,
    max = 1.0,
    step = 0.1,
    readout_format='.1f'
)

widget=interactive(plot_areas,
                   variable=variable,
                   colormap=colormap,
                   intervention_age=intervention_age,
                   therapeutic_factor=therapeutic_factor,
                   harmful_variable=harmful_variable
                   )
#creates a custom arragement of how the widgets are placed
controls = HBox((VBox((variable,intervention_age,therapeutic_factor)),
                VBox((harmful_variable,colormap)),
                VBox((widget.children[-3:-1]))))
output = widget.children[-1]
display(VBox([controls, output]))